    Ben Christensen
    Math 323
    March 13, 2018

Use the CVXOPT package to solve convex optimization problems.

In [1]:
import numpy as np
from cvxopt import matrix, solvers

In [2]:
solvers.options['show_progress'] = False
def prob1():
    """Solve the following convex optimization problem:

    minimize        2x + y + 3z
    subject to      x + 2y          >= 3
                    2x + 10y + 3z   >= 10
                    x               >= 0
                    y               >= 0
                    z               >= 0

    Returns (in order):
        The optimizer x (ndarray)
        The optimal value (sol['primal objective'])
    """
    #initializes matrix objects according to standard form of opt. problem
    c = matrix(np.array([2., 1., 3.]))
    G = matrix(np.array([[-1., -2., 0.],
                         [-2., -10., -3.],
                         [-1., 0., 0.],
                         [0., -1., 0.],
                         [0., 0., -1.]]))
    h = matrix(np.array([-3., -10., 0., 0., 0.]))
    #Solve
    sol = solvers.lp(c, G, h)
    return np.ravel(sol['x']), sol['primal objective']




# Problem 2
def l1Min(A, b):
    """Calculate the solution to the optimization problem

        minimize    ||x||_1
        subject to  Ax = b

    Parameters:
        A ((m,n) ndarray)
        b ((m, ) ndarray)

    Returns:
        The optimizer x (ndarray), without any slack variables u
        The optimal value (sol['primal objective'])
    """
    #Save shape of A
    m,n = A.shape
    #Initialize optimization problem in standard form
    c = matrix(np.concatenate((np.ones(n), np.zeros(n))))
    G = matrix(np.vstack((np.hstack((-np.eye(n), np.eye(n))),
                          np.hstack((-np.eye(n), -np.eye(n))))))
    h = matrix(np.zeros(2*n))
    A = matrix(np.hstack((np.zeros((m,n)), A)))
    b = matrix(b)
    #Solve
    sol = solvers.lp(c, G, h, A, b)

    return np.ravel(sol['x'])[n:], sol['primal objective']


# Problem 3
def prob3():
    """Solve the transportation problem by converting the last equality constraint
    into inequality constraints.

    Returns (in order):
        The optimizer x (ndarray)
        The optimal value (sol['primal objective'])
    """
    c = matrix([4., 7., 6., 8., 8., 9.])
    #We add two inequalities to G in order to remove one equality from A
    G = matrix(np.vstack((-1*np.eye(6),
                          [0., 1., 0., 1., 0., 1.],
                          [0., -1., 0., -1., 0., -1.])))
    h = matrix(np.append(np.zeros(6), [8., -8.]))
    A = matrix(np.array([[1., 1., 0., 0., 0., 0],
                         [0., 0., 1., 1., 0., 0.],
                         [0., 0., 0., 0., 1., 1.],
                         [1., 0., 1., 0., 1., 0.]]))
    b = matrix([7., 2., 4., 5.])

    #Solve
    sol = solvers.lp(c, G, h, A, b)

    return np.ravel(sol['x']), sol['primal objective']

# Problem 4
def prob4():
    """Find the minimizer and minimum of

    g(x,y,z) = (3/2)x^2 + 2xy + xz + 2y^2 + 2yz + (3/2)z^2 + 3x + z

    Returns (in order):
        The optimizer x (ndarray)
        The optimal value (sol['primal objective'])
    """
    #Initializ3e quadratic problem in standard form
    P = matrix(np.array([[3., 2., 1.], [2., 4., 2.], [1., 2., 3.]]))
    q = matrix([3., 0., 1.])
    sol = solvers.qp(P, q)

    return np.ravel(sol['x']), sol['primal objective']


# Problem 5
def l2Min(A, b):
    """Calculate the solution to the optimization problem

        minimize    ||x||_2
        subject to  Ax = b

    Parameters:
        A ((m,n) ndarray)
        b ((m, ) ndarray)

    Returns:
        The optimizer x (ndarray)
        The optimal value (sol['primal objective'])
    """
    n = A.shape[1]
    #We intialize P as follows because: 1/2*x.T(2I)x == x.Tx
    P = matrix(2*np.eye(n))
    # We need no second term in the quadratic function so q is the zero vector
    q = matrix(np.zeros(n))
    A = matrix(A)
    b = matrix(b)
    sol = solvers.qp(P, q, A=A, b=b)

    return np.ravel(sol['x']), sol['primal objective']


# Problem 6
def prob6():
    """Solve the allocation model problem in 'ForestData.npy'.
    Note that the first three rows of the data correspond to the first
    analysis area, the second group of three rows correspond to the second
    analysis area, and so on.

    Returns (in order):
        The optimizer x (ndarray)
        The optimal value (sol['primal objective']*-1000)
    """
    data = np.load("/Users/benchristensen/Desktop/ACME Python Labs/Volume2-Student-Materials/CVXOPT/ForestData.npy")
    #Initialize the problem using the data
    c = matrix(-data[:, 3])
    G = matrix(np.vstack((-data[:, 4],
                          -data[:, 5],
                          -data[:, 6],
                          -np.eye(21))))
    h = matrix(np.concatenate(([-40000., -5., -70.*788], np.zeros(21))))
    A = []
    b = []
    temp = data[:, 1]
    #Create the matrix A and sift through the data for b
    for i in range(7):
        A.append(np.insert(np.zeros(18), 3*i, [1., 1., 1.]))
        b.append(temp[3*i])
    A = matrix(np.array(A))
    b = matrix(b)
    #Solve the problem
    sol = solvers.lp(c, G, h, A, b)

    return np.ravel(sol['x']), sol['primal objective']*-1000

In [3]:
prob1()

(array([ -1.14760924e-09,   1.50000000e+00,   6.95277519e-11]),
 1.4999999996249473)

In [4]:
#Test problem 2
A = np.array([[1.044,1.45, 3.094,3.082,3.316],[4.996,4.643,0.154,2.802,3.045],[0.053,4.978,2.31,3.503,1.281],[1.109,1.428,0.262,4.956,1.096],[0.126,4.856,3.652,4.971,2.111]])
b = np.array([3.371,0.709,2.023,1.904,0.661])
print(l1Min(A,b))

(array([-4.33999389,  0.9652075 , -4.44256345, -0.04814237,  6.15080483]), 15.946712022839238)


In [5]:
prob3()

(array([  5.00000001e+00,   1.99999999e+00,  -7.02698725e-09,
          2.00000001e+00,  -5.44879646e-09,   4.00000001e+00]),
 85.99999998207508)

In [6]:
prob4()

(array([-1.5,  1. , -0.5]), -2.5)

In [7]:
prob6()

(array([  2.94268976e-08,   1.26029756e-07,   7.49999998e+01,
          8.99999993e+01,   2.44248068e-07,   4.77280841e-07,
          1.39999998e+02,   6.94010881e-07,   1.24406937e-06,
          3.52187543e-08,   2.64064527e-08,   5.99999999e+01,
          1.84043187e-07,   1.54000001e+02,   5.79999985e+01,
          5.96632456e-08,   6.81083767e-08,   9.79999999e+01,
          3.26969561e-08,   2.25240136e-08,   1.13000000e+02]),
 322514998.9206208)